In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
from h3 import h3
import json
from urllib.request import URLError, Request, urlopen
from itertools import combinations
from itertools import permutations
from dateutil import parser
from datetime import datetime, timedelta
import math
import networkx as nx

In [6]:
df = pd.read_csv('Data/LaGuardia_Dropoff_Final.csv')
columns = ['tpep_pickup_datetime', 'tpep_dropoff_datetime','passenger_count',\
           'trip_distance', 'pickup_longitude','pickup_latitude','dropoff_longitude', 'dropoff_latitude']
df = df[columns]
df.rename(columns={'tpep_pickup_datetime':'pickup_time',
       'tpep_dropoff_datetime':'dropoff_time'},inplace=True)
drop_index=df[(df.pickup_latitude==0)|(df.pickup_longitude==0)].index
df.drop(drop_index,inplace=True)
df['pickup_time'] = pd.to_datetime(df['pickup_time'])
df['dropoff_time'] = pd.to_datetime(df['dropoff_time'])
df['pickup_h3'] = df.apply(lambda x: h3.geo_to_h3(x['pickup_latitude'], x['pickup_longitude'], 10), axis=1)
df['dropoff_h3'] = df.apply(lambda x: h3.geo_to_h3(x['dropoff_latitude'], x['dropoff_longitude'], 8), axis=1)


start_date = '2016-05-17 00:00:00' # Start date with time
end_date = '2016-05-17 23:59:59' # End date with time
df=df[(df['pickup_time'] >= start_date) & (df['dropoff_time'] <= end_date)]


In [13]:
df.reset_index(drop=True,inplace=True)
df.head()

,index,pickup_time,dropoff_time,passenger_count,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,pickup_h3,dropoff_h3
0,249700,2016-05-17 00:08:21,2016-05-17 00:40:30,1.0,11.90,-73.994621,40.750729,-73.862144,40.768654,8a2a100d2cb7fff,882a100e25fffff
1,249701,2016-05-17 00:01:31,2016-05-17 00:09:31,1.0,1.70,-73.862709,40.768848,-73.867104,40.768772,8a2a100e2507fff,882a100e25fffff
2,249702,2016-05-17 00:12:41,2016-05-17 00:18:34,2.0,3.11,-73.894112,40.747944,-73.869995,40.765556,8a2a100c6127fff,882a100e25fffff
3,249703,2016-05-17 00:20:19,2016-05-17 00:20:46,1.0,0.00,-73.863472,40.769848,-73.863533,40.769802,8a2a100e2517fff,882a100e25fffff
4,249704,2016-05-17 00:36:23,2016-05-17 00:43:33,1.0,1.70,-73.862831,40.769043,-73.867058,40.768787,8a2a100e2507fff,882a100e25fffff


# Installation steps


### 1.Install the latest JRE and get GraphHopper Server as zip from <a href=https://graphhopper.com/public/releases/graphhopper-web-0.10.3-bin.zip>Graphhopper API</a>. Unzip it.
### 2.Copy this OSM file into the SAME unzipped directory: <a href=https://download.geofabrik.de/north-america/us/new-york-latest.osm.pbf >new-york-latest.osm.pbf</a>
### 3.Start GraphHopper Maps via: java -jar graphhopper-web-0.10.3-with-dep.jar jetty.resourcebase=webapp config=config-example.properties datareader.file=new-york-latest.osm.pbf. 
### 3.Test to see if its running after you see 'Started server at HTTP 8989' by going to http://localhost:8989/ and you should see a map of New York.
### 4.Keep this running when executing our program because this is the API

In [30]:
# Check how graphhopper works
a,b,c,d = df['pickup_latitude'].loc[1],df['pickup_longitude'].loc[1],df['dropoff_latitude'].loc[1],df['dropoff_longitude'].loc[1]
request_str = 'http://localhost:8989/route?point=' + str(a) + '%2C' + str(b) + '&point=' + str(c) + '%2C' + str(d) + '&vehicle=car'
request = Request(request_str)
res=requests.get(request_str)
print("Distance = {}".format(json.loads(res.text)['paths'][0]['distance']))
print("Time = {}".format(json.loads(res.text)['paths'][0]['time']))
# Distance = 1158.322
# Time = 128375

Distance = 1049.667
Time = 111745


# To calculate pool_window.
## The dataframe will be divided into pools.We run algorithm for each of these pools

In [15]:
def ceil_dt(dt, delta):
    return datetime.min + math.ceil((dt - datetime.min) / delta) * delta

pool_time_window = 10 # Change pool time window
df['pool_window'] = df['pickup_time'].apply(lambda x: ceil_dt(x.to_pydatetime(), timedelta(minutes=pool_time_window)))
df['duration'] = (df['pickup_time']-df['dropoff_time']).dt.seconds
df['delay'] = df['duration'].apply(lambda x: x*0.20)

## This is expected to be calculated and stored in a csv file. Later loaded into a dataframe before running the algorithm.The columns that should be stored in the csv files are same as following dataframe.

In [16]:
def get(dataframe):
    a,b,c,d=[],[],[],[]
    df_distance =  pd.DataFrame(columns = ['pickup_h3','dropoff_h3','distance','duration'])
    for node_a, node_b in list(permutations(dataframe.index, 2)):
        if node_a == 2069 and node_b==2070:print('yes')
        temp_curr, temp_next = [], []

        from_location = dataframe.iloc[node_a]['pickup_h3']
        to_location = dataframe.iloc[node_b]['pickup_h3']

        e, f, g, h = dataframe.iloc[node_a]['pickup_latitude'], dataframe.iloc[node_a]['pickup_longitude'],\
        dataframe.iloc[node_b]['pickup_latitude'],dataframe.iloc[node_b]['pickup_longitude']
        
        request_str = 'http://localhost:8989/route?point=' + str(e) + '%2C' + str(f) + '&point=' + str(
            g) + '%2C' + str(h) + '&vehicle=car'
        request = Request(request_str)
        res = requests.get(request_str)
        distance = json.loads(res.text)['paths'][0]['distance']

        time = json.loads(res.text)['paths'][0]['time']
            minute, msec = divmod(time, 60000)
            if (distance / 1609.344) <=3:
                a.append(from_location)
                b.append(to_location)
                c.append(distance / 1609.344)  # convert meters to miles
                d.append(minute + (msec / 100000))  # convert ms to s and add to min

    df_distance['pickup_h3'] = a
    df_distance['dropoff_h3'] = b
    df_distance['distance'] = c
    df_distance['duration'] = d
    return df_distance
# df_distance = df_distance.set_index(['pickup_h3','dropoff_h3'])
# df_distance.to_csv('29jan2016_30jan2016.csv') 

In [17]:
final_distance=[]
for _,trips in df.groupby(['pool_window']):
    trips = trips.reset_index()
    df_distance=  get(trips)
    final_distance.append(df_distance)
df_distance = pd.concat(final_distance)
df_distance.reset_index(drop=True,inplace=True)

In [19]:
df_distance.shape

(55814, 4)

# The nodes of the graph

In [20]:
class Node:
    def __init__(self,idx,data):
        self.id = idx
        self.pickup_location = (data.pickup_latitude,data.pickup_longitude,data.pickup_h3)
        self.dropoff_location = (data.dropoff_latitude,data.dropoff_longitude,data.dropoff_h3)
        self.pickup_time = data.pickup_time
        self.dropoff_time = data.dropoff_time
        self.distance = data.trip_distance
        self.duration = data.duration
        self.delay = data.delay
        #TBD..

# Lets try with a sample of 4 trips going to LGA

## Creating a toy distance df for the above sample dataframe.Note: Since the 'Data/LaGuardia_Dropoff_Final.csv' corresponds to LGA as destination we need only the distance between the pickup of individual trips.

## For trips going from LGA we would be needing distance between dropoffs

In [21]:
# df_distance =  pd.read_csv('1_29_to_1_30.csv')
df_distance.groupby(['pickup_h3','dropoff_h3']).count()
df_distance.drop_duplicates(subset=['pickup_h3','dropoff_h3'],keep=False,inplace=True)
df_distance.set_index(['pickup_h3','dropoff_h3'])
df_distance = df_distance.sort_index()
df_distance.head()

,pickup_h3,dropoff_h3,distance,duration
0,8a2a100d2cb7fff,8a2a100e2507fff,8.746133,14.05751
1,8a2a100e2507fff,8a2a100d2cb7fff,9.921277,14.12815
3,8a2a100e2517fff,8a2a103b0217fff,11.781887,13.20557
4,8a2a100e2517fff,8a2a100f1bb7fff,1.175470,2.34617
6,8a2a100e2507fff,8a2a103b0217fff,11.850976,13.30178


In [22]:
# ('8a2a1008891ffff', '8a2a100d6a97fff') in df_distance.index
df_distance.shape
# cn
# request_str = 'http://localhost:8989/route?point=' + str(40.80505) + '%2C' +str(-73.966179)  + '&point=' +  str(40.758911)+ '%2C' + str(-73.968277) + '&vehicle=car'
# request = Request(request_str)
# res=requests.get(request_str)
# print("Distance = {}".format(json.loads(res.text)['paths'][0]['distance']))
# print("Time = {}".format(json.loads(res.text)['paths'][0]['time']))

(43068, 4)

In [23]:
def get_distance_duration(node_a,node_b,trip_type):
    if trip_type==2: 
        e, f, g, h = node_a.pickup_location[0], node_a.pickup_location[1], node_b.pickup_location[0],node_b.pickup_location[1]
    else:
        pass
        #TBD
    request_str = 'http://localhost:8989/route?point=' + str(e) + '%2C' + str(f) + '&point=' + str(
        g) + '%2C' + str(h) + '&vehicle=car'
    request = Request(request_str)
    res = requests.get(request_str)
    distance = json.loads(res.text)['paths'][0]['distance']
    time = json.loads(res.text)['paths'][0]['time']
    minute, msec = divmod(time, 60000) 
    return distance / 1609.344 , minute + (msec / 100000)

In [24]:
def get_all_pairs(node_a,node_b,trip_type):
    if trip_type == 1:
        #Combination LGA--> a -->b
        #if no distance call graphhopper 
        LGA_a_dist = df_distance.loc[(node_a.pickup_location[2],node_a.dropoff_location[2])]['distance']
        a_b_dist = df_distance.loc[(node_a.dropoff_location[2],node_b.dropoff_location[2])]['distance']
        LGA_a_dur = df_distance.loc[(node_a.pickup_location[2],node_a.dropoff_location[2])]['duration']
        a_b_dur = df_distance.loc[(node_a.dropoff_location[2],node_b.dropoff_location[2])]['duration']
        
        #Combination LGA--> b -->a
        LGA_b_dist = df_distance.loc[(node_b.pickup_location[2],node_b.dropoff_location[2])]['distance']
        b_a_dist = df_distance.loc[(node_b.dropoff_location[2],node_a.dropoff_location[2])]['distance']
        LGA_b_dur = df_distance.loc[(node_b.pickup_location[2],node_b.dropoff_location[2])]['duration']
        b_a_dur = df_distance.loc[(node_b.dropoff_location[2],node_a.dropoff_location[2])]['duration']
        
        path_1_total_dis,path_1_total_dur = LGA_a_dist + a_b_dist,LGA_a_dur + a_b_dur 
        path_1_a_dur,path_1_b_dur = LGA_a_dur,path_1_total_dur
        
        path_2_total_dis,path_2_total_dur = LGA_b_dist+b_a_dist,LGA_b_dur+b_a_dur
        path_2_a_dur,path_2_b_dur         = path_2_total_dur ,LGA_b_dur
               
    else:
        
        if (node_a.pickup_location[2],node_b.pickup_location[2]) not in df_distance.index:
            a_b_distance,a_b_duration = get_distance_duration(node_a,node_b,2)
        else:
            a_b_distance = df_distance.loc[(node_a.pickup_location[2],node_b.pickup_location[2])]['distance']
            a_b_duration = df_distance.loc[(node_a.pickup_location[2],node_b.pickup_location[2])]['duration']
        
        #Combination a--> b --> LGA
        a_b_dist   = a_b_distance
        b_LGA_dist = node_b.distance 
        a_b_dur    = a_b_duration
        b_LGA_dur  = node_b.duration
        
        if (node_b.pickup_location[2],node_a.pickup_location[2]) not in df_distance.index:
            b_a_distance,b_a_duration = get_distance_duration(node_b,node_a,2)
        else:
            b_a_distance = df_distance.loc[(node_b.pickup_location[2],node_a.pickup_location[2])]['distance']
            b_a_duration = df_distance.loc[(node_b.pickup_location[2],node_a.pickup_location[2])]['duration']
        
        #Combination b--> a --> LGA
        b_a_dist   = b_a_distance
        a_LGA_dist = node_a.distance 
        b_a_dur    = b_a_duration
        a_LGA_dur  = node_a.duration
        
        path_1_total_dis,path_1_total_dur = a_b_dist + b_LGA_dist,a_b_dur + b_LGA_dur 
        path_1_a_dur,path_1_b_dur = path_1_total_dur,b_LGA_dur
        
        path_2_total_dis,path_2_total_dur, = b_a_dist+a_LGA_dist,b_a_dur+a_LGA_dur
        path_2_a_dur,path_2_b_dur         = a_LGA_dur,path_2_total_dur
        
    return ((path_1_total_dis,path_1_total_dur,path_1_a_dur,path_1_b_dur),( path_2_total_dis,path_2_total_dur,path_2_a_dur,path_2_b_dur))
    

In [25]:
def calculate_edge_weight(node_a,node_b):
    path1,path2 = get_all_pairs(node_a,node_b,2)
    minimum_distance = float('inf')
    for path in (path1,path2):
        distance_contraint = (path[0] <= node_a.distance + node_b.distance)
        delay_constraint = (path[2] <= node_a.duration + node_a.delay) & (path[3] <= node_b.duration + node_b.delay)
        #add social constraint too...
        
        
        if distance_contraint and delay_constraint and path[0]< minimum_distance:
            minimum_distance = path[0]
    distance_saved = node_a.distance + node_b.distance - minimum_distance
    return distance_saved

In [26]:
def get_rsg(G):
    for node_a,node_b in list(combinations(G,2)):
        distance_saved = calculate_edge_weight(node_a,node_b)
        if distance_saved!= float('inf'):
            G.add_edge(node_a,node_b, weight=distance_saved)
    return G

# Running the algorithm for sample dataframe

In [27]:
graphs = []
for _,trips in df.groupby(['pool_window']):
    nodes = []
    trips = trips.reset_index()
    for idx, row in trips.iterrows():
        nodes.append(Node(idx,trips.iloc[idx]))
    G = nx.Graph()
    G.add_nodes_from(nodes)
    graphs.append(G)
    
#Start of the code
weight_matches = []
cn=0
for g in graphs:
    ride_sharing_graph = get_rsg(g)
    #maximum weighted algorithm
    match = nx.max_weight_matching(ride_sharing_graph, maxcardinality=True)
    g_match = nx.Graph()
    for u,v in match:
        g_match.add_edge(u,v)
        
    weight_matches.append(g_match)

/home/shet/.local/lib/python3.7/site-packages/networkx/algorithms/matching.py:370: RuntimeWarning: invalid value encountered in double_scalars
  return dualvar[v] + dualvar[w] - 2 * G[v][w].get(weight, 1)


In [28]:
print("Selected edges by maximum weight algorithm")
for u,v in weight_matches[0].edges:
    print(u.id,v.id,ride_sharing_graph.get_edge_data(u,v))

Selected edges by maximum weight algorithm
1 0 None
